# Tarea 2: Piping y Bombas
### Pablo Correa e Ian Gross

## Librerías

In [10]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte
import matplotlib.pyplot as plt

from fluids.units import *

In [11]:
g = cte.g*u.m/u.s**2

## Problema 1

Considere el sistema de distribución de agua en la figura, donde la presión en A es 150 psig.

Considerando las tuberías de entrada y salida 1 y 2 respectivamente, y las tuberías paralelas A, B y C desde arriba hacia abajo, se tiene que:

In [19]:
# datos
z_A = (100*u.ft).to(u.m)
z_B = (120*u.ft).to(u.m)

L_1 = (2000*u.ft).to(u.m)
L_2 = (4000*u.ft).to(u.m)
L_A = (3000*u.ft).to(u.m)
L_B = (2000*u.ft).to(u.m)
L_C = (3000*u.ft).to(u.m)

D_1 = (24*u.inch).to(u.m)
D_2 = (30*u.inch).to(u.m)
D_A = (14*u.inch).to(u.m)
D_B = (12*u.inch).to(u.m)
D_C = (16*u.inch).to(u.m)

A_1 = np.pi * D_1**2 / 4
A_2 = np.pi * D_2**2 / 4
A_A = np.pi * D_A**2 / 4
A_B = np.pi * D_B**2 / 4
A_C = np.pi * D_C**2 / 4

Q = (20*u.cubic_feet/u.s).to(u.m**3/u.s)
P_1 = ((150 + 14.7)*u.psi).to(u.Pa) # psig = psi + 14.7
T = 300*u.K # Asumiendo temperatura ambiente
epsilon = (0.0015*u.mm).to(u.m) # Asumiendo tubería de PVC [1]

rho = cp.PropsSI('D', 'P', P_1.magnitude, 'T', T.magnitude, 'Water')*u.kg/u.m**3
mu = cp.PropsSI('V', 'P', P_1.magnitude, 'T', T.magnitude, 'Water')*u.Pa*u.s

a. La pérdida de carga desde A a B, si es que solo la tubería A es utilizada.

Considerando que solo se usa la tubería A, los caudales serían:

$$\dot{\forall} = \dot{\forall}_1 = \dot{\forall}_2 = \dot{\forall}_A$$
$$\dot{\forall}_B = \dot{\forall}_C = 0$$

Además, se sabe que las pérdidas de carga corresponderían a las pérdidas por fricción en los tramos y no hay singularidades, por lo tanto:

$$\Delta P = \Delta P_1 + \Delta P_2 + \Delta P_A$$
$$\Delta P_i = f_i \rho \frac{L_i}{D_i} \frac{V_i^2}{2}$$

Siendo $V_i$ la velocidad promedio en el tramo. Suponiendo que el caudal es constante a lo largo del tramo, y que las tuberías tienen perfil circular, se puede estimar que las velocidades serían:

$$V_i = \frac{\dot{\forall}_i}{A_i} = \frac{4 \dot{\forall}}{\pi D_i^2}$$

Con esta expresión, la pérdida total sería

$$\Delta P = \frac{\rho}{2} (f_1 \frac{L_1}{D_1} V_1^2 + f_2 \frac{L_2}{D_2} V_2^2 + f_A \frac{L_A}{D_A} V_A^2)$$

In [20]:
v_1 = Q / A_1
v_2 = Q / A_2
v_A = Q / A_A

Re_1 = fld.Reynolds(D=D_1, rho=rho, mu=mu, V=v_1)
Re_2 = fld.Reynolds(D=D_2, rho=rho, mu=mu, V=v_2)
Re_A = fld.Reynolds(D=D_A, rho=rho, mu=mu, V=v_A)

f_1 = fld.friction.friction_factor(Re_1, eD=epsilon/D_1)
f_2 = fld.friction.friction_factor(Re_2, eD=epsilon/D_2)
f_A = fld.friction.friction_factor(Re_A, eD=epsilon/D_A)

DP_1 = rho/2*(f_1*L_1/D_1*v_1**2 + f_A*L_A/D_A*v_A**2 + f_2*L_2/D_2*v_2**2 + g*(z_B - z_A))
print(f"La pérdida de carga total es {DP_1.to(u.kPa)}")

La pérdida de carga total es 492.57903720404875 kilopascal


b. La pérdida de carga desde A a B, si es que las tuberías A y B son utilizadas.

Considerando que se usan las tuberías A y B, se puede suponer que los caudales tienen las siguientes relaciones:

$$\dot{\forall} = \dot{\forall}_A + \dot{\forall}_B$$
$$\dot{\forall}_C = 0$$

Por otro lado, como las tuberías se encuentran paralelas se sabe que:

$$\Delta P_A = \Delta P_B$$
$$\Delta P_A - \Delta P_B = 0$$

Reemplazando las expresiones de velocidad y presión de la parte (a.) se tiene que:

$$A_A V_A + A_B V_B = \dot{\forall}$$
$$f_A \frac{\rho}{2} \frac{L_A}{D_A} V_A^2 - f_B \frac{\rho}{2} \frac{L_B}{D_B} V_B^2 = 0$$

Se da un ansatz de la distribución inicial de caudales. Luego, se itera sobre estos valores hasta llegar a un resultado parejo.

In [26]:
# Ansatz e iteración manual. Asegurando la conservación del caudal.
x = 0.546366
Q_A = x*Q
Q_B = (1-x)*Q

v_1 = Q / A_1
v_2 = Q / A_2
v_A = Q_A / A_A
v_B = Q_B / A_B

Re_1 = fld.Reynolds(D=D_1, rho=rho, mu=mu, V=v_1)
Re_2 = fld.Reynolds(D=D_2, rho=rho, mu=mu, V=v_2)
Re_A = fld.Reynolds(D=D_A, rho=rho, mu=mu, V=v_A)
Re_B = fld.Reynolds(D=D_B, rho=rho, mu=mu, V=v_B)

f_1 = fld.friction.friction_factor(Re_1, eD=epsilon/D_1)
f_2 = fld.friction.friction_factor(Re_2, eD=epsilon/D_2)
f_A = fld.friction.friction_factor(Re_A, eD=epsilon/D_A)
f_B = fld.friction.friction_factor(Re_B, eD=epsilon/D_B)

# Se debe dar igualdad de presiones para A y B
P_A = rho*f_A*L_A/D_A*v_A**2/2
P_B = rho*f_B*L_B/D_B*v_B**2/2

print(f"La pérdida de carga en A es {P_A.to(u.Pa)}")
print(f"La pérdida de carga en B es {P_B.to(u.Pa)}")
print(f"La diferencia de presiones es {(P_A - P_B).to(u.Pa)}")

La pérdida de carga en A es 140123.58084382655 pascal
La pérdida de carga en B es 140123.42375500116 pascal
La diferencia de presiones es 0.15708882539183833 pascal


In [22]:
# Sumando las pérdidas de carga 1 y 2 y la diferencia de alturas
DP_2 = rho/2*(f_1*L_1/D_1*v_1**2 + f_A*L_A/D_A*v_A**2 + f_2*L_2/D_2*v_2**2 + g*(z_B - z_A))
print(f"La pérdida de carga total es {DP_2.to(u.kPa)}")

La pérdida de carga total es 204.89246654202438 kilopascal


c. La pérdida de carga desde A a B, si es que las 3 tuberías son utilizadas.

Considerando ahora todas las tuberías, se tiene que:

$$\dot{\forall} = \dot{\forall}_A + \dot{\forall}_B + \dot{\forall}_C$$
$$\Delta P_A = \Delta P_B = \Delta P_C$$

Por lo tanto:

$$\dot{\forall}_A + \dot{\forall}_B + \dot{\forall}_C = \dot{\forall}$$
$$\Delta P_A - \Delta P_B = 0$$
$$\Delta P_A - \Delta P_C = 0$$

Reemplazando las expresiones, se tiene que:

$$A_A V_A + A_B V_B + A_C V_C = \dot{\forall}$$
$$f_A \frac{\rho}{2} \frac{L_A}{D_A} V_A^2 - f_B \frac{\rho}{2} \frac{L_B}{D_B} V_B^2 = 0$$
$$f_A \frac{\rho}{2} \frac{L_A}{D_A} V_A^2 - f_C \frac{\rho}{2} \frac{L_C}{D_C} V_B^2 = 0$$

In [25]:
# Ansatz e iteración manual. Asegurando la conservación del caudal.
x = 0.3073715
y = 0.255237
Q_A = x*Q
Q_B = y*Q
Q_C = (1-x-y)*Q

v_1 = Q / A_1
v_2 = Q / A_2
v_A = Q_A / A_A
v_B = Q_B / A_B
v_C = Q_C / A_C


Re_1 = fld.Reynolds(D=D_1, rho=rho, mu=mu, V=v_1)
Re_2 = fld.Reynolds(D=D_2, rho=rho, mu=mu, V=v_2)
Re_A = fld.Reynolds(D=D_A, rho=rho, mu=mu, V=v_A)
Re_B = fld.Reynolds(D=D_B, rho=rho, mu=mu, V=v_B)
Re_C = fld.Reynolds(D=D_C, rho=rho, mu=mu, V=v_C)

f_1 = fld.friction.friction_factor(Re_1, eD=epsilon/D_1)
f_2 = fld.friction.friction_factor(Re_2, eD=epsilon/D_2)
f_A = fld.friction.friction_factor(Re_A, eD=epsilon/D_A)
f_B = fld.friction.friction_factor(Re_B, eD=epsilon/D_B)
f_C = fld.friction.friction_factor(Re_C, eD=epsilon/D_C)

# Se debe dar igualdad de presiones para A y B
P_A = rho*f_A*L_A/D_A*v_A**2/2
P_B = rho*f_B*L_B/D_B*v_B**2/2
P_C = rho*f_C*L_C/D_C*v_C**2/2

print(f"La presión en A es {P_A.to(u.kPa)}")
print(f"La presión en B es {P_B.to(u.kPa)}")
print(f"La presión en C es {P_C.to(u.kPa)}\n")

print(f"La diferencia de presiones A-B es {(P_A - P_B).to(u.Pa)}")
print(f"La diferencia de presiones A-C es {(P_A - P_C).to(u.Pa)}")
print(f"La diferencia de presiones B-C es {(P_B - P_C).to(u.Pa)}")

La presión en A es 48.76796414776605 kilopascal
La presión en B es 48.76769734585945 kilopascal
La presión en C es 48.76797488749972 kilopascal

La diferencia de presiones A-B es 0.266801906596811 pascal
La diferencia de presiones A-C es -0.010739733668742701 pascal
La diferencia de presiones B-C es -0.2775416402655537 pascal


In [17]:
# Sumando las pérdidas de carga 1 y 2 y la diferencia de alturas
DP_3 = rho/2*(f_1*L_1/D_1*v_1**2 + f_A*L_A/D_A*v_A**2 + f_2*L_2/D_2*v_2**2 + g*(z_B - z_A))
print(f"La pérdida de carga total es {DP_3.to(u.kPa)}")

La pérdida de carga total es 0.006015697539296102 kilopascal


d. Determine la potencia requerida (en caballos de fuerza) para hacer la presión en B sea igual
a A, en los 3 ítems anteriores.

In [18]:
P_B_1 = P_A - DP_1
P_B_2 = P_A - DP_2
P_B_3 = P_A - DP_3

hp_1 = P_B_1*Q
hp_2 = P_B_2*Q
hp_3 = P_B_3*Q

print(f"La presión en B calculada con el primer método es {P_B_1}")
print(f"La presión en B calculada con el segundo método es {P_B_2}")
print(f"La presión en B calculada con el tercer método es {P_B_3}\n")

print(f"Caballos de potencia requeridos con el primer método es {hp_1.to(u.hp)}")
print(f"Caballos de potencia requeridos con el segundo método es {hp_2.to(u.hp)}")
print(f"Caballos de potencia requeridos con el tercer método es {hp_3.to(u.hp)}")

La presión en B calculada con el primer método es -10.980165488000322 pascal
La presión en B calculada con el segundo método es -5.134583309240455 pascal
La presión en B calculada con el tercer método es -2.0548714930590743 pascal

Caballos de potencia requeridos con el primer método es -0.008339109969772199 horsepower
Caballos de potencia requeridos con el segundo método es -0.003899563709809885 horsepower
Caballos de potencia requeridos con el tercer método es -0.001560613942758556 horsepower


e. A partir de los resultados obtenidos en (d.) ¿Que puede concluir a partir de a utilización de
tubería sen paralelos para sistemas de bombeo?

Las tuberías en paralelo implican que cada tramo de tubería tendrá un menor caudal y, por lo tanto, una menor pérdida por fricción.

**Tiene que haber una desventaja. La falta de presión en las tuberías individuales puede hacer que el flujo deje de correr??**

f. Considerando una bomba modelo B3Z-M en el tramo 1 (L=2000ft y D=24in), con un
rodete de 8,438” (Página 45 del catálogo Berkeley). Determinar el caudal de operación en cada
tramo de tubería.

g. Considerando la misma bomba usada en (f) pero instalada en el tramo central (L=2000ft y
D=24in). Determinar la distriubución del caudal de operación.

## Problema 2

A Bell & Gossett Series e-1531 end-suction pump is to be selected for the system
shown in Figure P4.42. The system is designed to move water at 60°F from the
lower tank to the upper tank. The system must deliver a minimum of 500 gpm of
water from the lower tank to the upper tank when the pump is operating. A tilting
disc check valve (α = 5°) is used to prevent the water from flowing backward
when the pump is shut off. The total length of straight pipe in this system is
150 ft. The pipe is 4-nom sch 40 commercial steel.

a. Plot the system curve up to 800 gpm.

b. Select a Series e-1531 Bell & Gossett pump for this system.

c. What is the operating point of the pump/pipe system based on the pump you
selected in (b)? Specify the capacity (gpm), head (ft), brake horsepower input
to the pump (hp), and efficiency of the pump.

## Problema 3

A two-tank pump/pipe system has a head requirement of 23.4 ft at a flow rate of
100 gpm. The elevation difference between the fluid surfaces in the tanks is 10 ft.
The pump is a Bell & Gossett Series e-80 Model 2.5x2.5x9C operating at 1170 rpm
with a 7.5-inch impeller.

a. Develop an equation for the system curve.

b. Use a computer program to develop an equation for the 7.5-inch impeller
using data read from the pump performance curve.

c. Determine the capacity (gpm) and head (ft) of the pump/pipe system by
simultaneously solving the pump curve and system equations.

d. Use the pump performance curve to determine the power draw (hp) and efficiency
of the pump.

## Problema 4

Considere el plano hidráulico de la localidad de Hualañé. Establezca un código computracional que
permita optimizar lls diámetros de tuberñia de la red hidráulica considerada en la Tarea 1. El objetivo de la optimización es que la diferencia de presiones enre las acometidas de las casas sea lo menor posible,
siempre considerando que la presión debe ser mayor a lo establecido por la nomrma de sistemas sanitarios. 

Considerando el caso extremo, cuando todas las casas tienen una llave de lavamanos abierta, determine el cabezal que debería sumiinistrar el sistema de bombreo.

## Referencias
[1] https://www.pipeflow.com/pipe-pressure-drop-calculations/pipe-roughness
